<a href="https://colab.research.google.com/github/JuanPabl07DP/Cardiovascular_FNN/blob/main/Cardiovarcular_FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cardiovascular-disease-dataset


In [10]:
import pandas as pd

df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [11]:
import numpy as np

# Eliminar columnas innecesarias (ID si existe)
df = df.drop(columns=['id'], errors='ignore')

# Convertir 'gender' a binario (0 = mujer, 1 = hombre)
df['gender'] = df['gender'].map({1: 1, 2: 0})

# Revisar valores extremos
df = df[(df['height'] > 100) & (df['height'] < 250)]
df = df[(df['weight'] > 30) & (df['weight'] < 200)]
df = df[(df['ap_hi'] > 70) & (df['ap_hi'] < 250)]
df = df[(df['ap_lo'] > 40) & (df['ap_lo'] < 200)]

# Normalización
from sklearn.preprocessing import MinMaxScaler

features = df.drop(columns=['cardio'])
labels = df['cardio']

scaler = MinMaxScaler()
X = scaler.fit_transform(features)
y = labels.values

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

In [13]:
from sklearn.model_selection import train_test_split

# Primero 85% para train+dev y 15% para test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Ahora dividir el 85% restante en 70% train y 15% dev (≈ 0.176 de 85%)
X_train, X_dev, y_train, y_dev = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

print(f'Train: {len(X_train)} samples')
print(f'Dev: {len(X_dev)} samples')
print(f'Test: {len(X_test)} samples')

Train: 48119 samples
Dev: 10279 samples
Test: 10306 samples


In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(256, input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.4),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_dev_scaled, y_dev),
                    epochs=40,
                    batch_size=64,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)


Epoch 1/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7103 - loss: 0.7884 - val_accuracy: 0.7300 - val_loss: 0.6385 - learning_rate: 5.0000e-04
Epoch 2/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7271 - loss: 0.6249 - val_accuracy: 0.7258 - val_loss: 0.5902 - learning_rate: 5.0000e-04
Epoch 3/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7371 - loss: 0.5798 - val_accuracy: 0.7284 - val_loss: 0.5703 - learning_rate: 5.0000e-04
Epoch 4/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7332 - loss: 0.5687 - val_accuracy: 0.7278 - val_loss: 0.5633 - learning_rate: 5.0000e-04
Epoch 5/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7359 - loss: 0.5606 - val_accuracy: 0.7289 - val_loss: 0.5587 - learning_rate: 5.0000e-04
Epoch 6/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7310 - loss: 0.5595 - val_accuracy: 0.7286 - val_loss: 0.5569 - learning_rate: 5.0000e-04
Epoch 7/40
752/752 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 

In [18]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7295 - loss: 0.5476
Test Accuracy: 0.7296, Test Loss: 0.5501


In [19]:
model.save("modelo_cardio_hibrido_fnn.h5")